# This is a note for [Camera Parameters - Extrinsics and Intrinsics (Cyrill Stachniss, 2020)](https://youtu.be/uHApDqH-8UE)

Personally, I prefer to setup the camera/image/sensor coordinate system as shown in the following [image](https://learnopencv.com/geometry-of-image-formation/). And I will use these coordinate system in this notebook.
![camera](./camera.png)

The ones used in the video are a bit confusing because there are two ways of forming the camera coordinate system in the video. One is at 6:20 and the image plane is behind the camera origin. The other one is at 23:19 and the z-axis is pointing away from the outside world. P.S. any of these two is good if you work out the math properly. But this is just a personal preference.

There are 4 coordinate system: world (3D) <-> camera (3D) -> image (2D) <-> sensor(2D).

### world (3D) <-> camera (3D):
This is a revertible rigid body transform.

As a convention, the origin is at the pin hole, the image plane is in front of the pin hole instead of behind. This way, you do not need to flip the image. The z-axis is where the camera is pointing at. The x-axis and y-axis are pointing at the same direction of the x-axis and y-axis in the image coordinate system as well as the u and v in sensor coordinate system.

### camera (3D) -> image (2D):
This is an centeral projection.

In the camera coordinate system, there is a plane at z = f. This plane is the image coordinate system. This is why its x and y axis have the same direction as camera coordinate system.

x_image = x_camera / z_camera * f

y_image = y_camera / z_camera * f


### image (2D) <-> sensor(2D):
This is revertible transformation that may contain scale (x, y seperately), tranlation, skew.

Since there is no rotation, the u and v have the same direction as x and y axis in image/camera coordinate. As a convension, u is point horizontally to the right in the image, and v is pointing vertically down.
